# <center>Quantum Steganography</center>

# 1. Introduction to Quantum Key Distribution

<b>"Key distribution"</b> is a type of secure communication protocol which is accomplished with the use of a <b>"key"</b> (which is just a bitstring with a sequence of 0s and 1s) that is uniquely known only by the sender (let's call them "Alice") and receiver (let's call them "Bob"), and enables them to encrypt and decrypt a sent message. If a key can be securely distributed between Alice and Bob, the encrypted message can be securely sent over a public channel. Without this key, it is difficult to sucessfully decrypt this message by an eavesdropper. Thus, a secure communication relies on the ability of Alice to encrypt the message in a way that only Bob can decrypt it. 

<b>"Quantum key distribution"</b> or <b>"QKD"</b> is unique compared to classical key distribution schemes because it makes use of an important aspect of quantum mechanics where any measurement performed on a quantum system (by any third party, in this case an eavesdropper) disturbs it, thus detecting eavesdropping attempts and securing the message.

We will be using <b>Qiskit</b> code to implement <b>BB84 protocol</b> which is a very popular quantum key distribution scheme.


## 1.1 QKD Layout 

Implementation of the BB84 protocol by following these steps:
<br>
>1.<b> SELECT ENCODING</b>: Alice randomly selects a basis ( × or + ) to encode each bit.<br>
    2.<b> SELECT MEASUREMENT</b>: Bob randomly selects a basis ( × or + ) to measure each bit.<br>
    3.<b> ENCODING</b>: Alice creates the quantum states encoded in the selected bases.<br>
    4.<b> SENDING</b>: Alice sends Bob the encoded states via the quantum channel.<br>
    5.<b> MEASUREMENT</b>: Bob measures the quantum states in his pre-selected measurement bases.<br>
    6.<b> SEND BASES</b>: Alice send which basis were used to encode each bit via the classical channel.<br>
    7.<b> FIND SYMMETRIC KEY</b>: Alice and Bob discard bits in their key that used a different encoding and decoding basis.<br>
 
These steps allow a secure key distribution between Alice and Bob, where the two will be able to send secure and encrypted messages to each other.

## 1.2 Imports and Installation


In [ ]:
!pip install qiskit==0.20.0
from IPython.display import clear_output
clear_output()

In [1]:
from random import getrandbits, choice
from qiskit import QuantumCircuit, Aer, execute

## 1.3 Protocol Requirements

Wwe will use a key length of 500.

In a real implemenetation of BB84, physical photons are sent through an optical fibre to Bob in step 4 of the QKD layout. Instead, we will be using <b>Quantum_Channel</b> as our optical fibre, which is simply a python list here.

Similarly, a classical channel is required, which is a wire that sends electrical signals. <b>Classical_Channel</b> is another list which we will be using here as a replacement.

In [2]:
key_length = 500
QUANTUM_CHANNEL = []
CLASSICAL_CHANNEL = []

# 2. Following the QKD steps

## 2.1 Step 1: Select Encoding

Alice randomly selects a bit key and a basis to encode each bit with the key.

The function given takes an argument as the number of bases that Alice needs to randomly select, and returns a bitstring of each selected encoding represented by either 0 or 1.

In [3]:
def select_encoding(length):
    
    #This stores the states Alice will encode
    alice_bitstring = ""
    # This stores the bases that Alice will prepare the states in
    alice_bases = ""
    
    # For the length 
    for i in range(length):
        # We use the function getrandbits to get either a 0 or 1 randomly,
        # The "1" in the function argument is the number of bits to be generated
        alice_bitstring += (str(getrandbits(1)))
        # 0 means encode in the (0,1) basis and 1 means encode in the (+,-) basis
        alice_bases += (str(getrandbits(1)))
    
    # return the string of bits and the list of bases they should be encoded in
    return alice_bitstring, alice_bases

Next we create the functions **alice_bitstring** and **alice_bases**. We can look at the first 10 elements of each, which should be in 0s and 1s.

In [4]:
alice_bitstring, alice_bases = select_encoding(key_length)

# Preview the first 10 elements of each:
print("alice_bitstring: ", alice_bitstring[:10])
print("alice_bases: ", alice_bases[:10])

alice_bitstring:  1111011010
alice_bases:  1100111010


## 2.2 Step 2: Select Measurement

Bob randomly selects a basis for measuring each bit. The below given function takes an argument as the number of bases that Bob needs to randomly select, and returns a bitstring of each chosen measurement basis represented by either 0 or 1.

In [5]:
def select_measurement(length):
    # Similar to before we store the bases that Bob will measure in a list
    bob_bases = ""
    
    for i in range(length):
        # Again we use getrandbits to generate a 0 or 1 randomly
        bob_bases += (str(getrandbits(1)))
        
    # return the list of random bases to measure in
    return bob_bases

In [6]:
bob_bases = select_measurement(key_length)

# Preview the first 10 elements of each:
print("selected_measurements: ", bob_bases[:10])

selected_measurements:  1011000100


## 2.3 Step 3: Encoding

After this, we will use functions **alice_bitstring** and **alice_bases** to generate the quantum states for Alice. We will use <b>QuantumCircuit</b> object to define our circuit and write the code to encode our bits, as demonstrated in the following <br>

| Bit in Alice's `alice_bitstring` | Corresponding bit in `alice_bases` | Encoding basis | Qubit state sent |
|:----------------:|:--------------------------:|:--------------------------:|:---------------:|
| 0 | 0 | $$|0\rangle,|1\rangle$$ |$$|0\rangle$$ |
| 0 | 1 | $$|+\rangle,|-\rangle$$ |$$|+\rangle$$ |
| 1 | 0 | $$|0\rangle,|1\rangle$$ |$$|1\rangle$$ |
| 1 | 1 | $$|+\rangle,|-\rangle$$ |$$|-\rangle$$ |

In [7]:
def encode(alice_bitstring, alice_bases):
    encoded_qubits = []
    for i in range(len(alice_bitstring)):
        # create a brand new quantum circuit called qc. Remember that the qubit will be in state |0> by default
        qc = QuantumCircuit(1,1)

        if alice_bases[i] == "0":
            # 0 Means we are encoding in the z basis
            if alice_bitstring[i] == "0":
                # We want to encode a |0> state, as states are intialized
                # in |0> by default we don't need to add anything here
                pass
            
            elif alice_bitstring[i] == "1":
                # We want to encode a |1> state
                # We apply an X gate to generate |1>
                qc.x(0)
                
        elif alice_bases[i] == "1":
            # 1 Means we are encoding in the x basis
            if alice_bitstring[i] == "0":
                # We apply an H gate to generate |+>
                qc.h(0)
            elif alice_bitstring[i] == "1":
                # We apply an X and an H gate to generate |->
                qc.x(0)
                qc.h(0)
            
        # add this quantum circuit to the list of encoded_qubits
        encoded_qubits.append(qc)
        
    return encoded_qubits

In [8]:
encoded_qubits = encode(alice_bitstring, alice_bases)

## 2.4 Step 4: Sending

Next, Alice sends the qubits_encoded to Bob via the **Quantum_Channel** function.

In [9]:
QUANTUM_CHANNEL = encoded_qubits

## 2.5 Step 5: Measurement

Now that Bob has received the qubits via **Quantum_Channel**, he measures the qubits on the measurement basis from step 2.

In [10]:
def measure(bob_bases, encoded_qubits, backend):
    # Perform measurement on the qubits send by Alice
    # selected_measurements: 
    # encoded_qubits: list of QuantumCircuits received from Alice
    # backend: IBMQ backend, either simulation or hardware
    
    # Stores the results of Bob's measurements
    bob_bitstring = ''
    
    for i in range(len(encoded_qubits)):
        qc = encoded_qubits[i]
        
        if bob_bases[i] == "0":
            # 0 means we want to measure in Z basis
            qc.measure(0,0)

        elif bob_bases[i] == "1":
            # 1 means we want to measure in X basis
            qc.h(0)
            qc.measure(0,0)
        
        # Now that the measurements have been added to the circuit, let's run them.
        job = execute(qc, backend=backend, shots = 1) # increase shots if running on hardware
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)

        # Append measured bit to Bob's measured bitstring
        bob_bitstring += measured_bit 
        
    return bob_bitstring

Measure the qubits that Bob received on the **qasm_simulator** backend. 

In [11]:
sim_backend = Aer.get_backend('qasm_simulator')

bob_bitstring = measure(bob_bases, QUANTUM_CHANNEL, sim_backend)

## 2.6 Step 6: Send Bases to Bob

Alice shares the bases that were used to encode each bit via a classical channel.

In [12]:
CLASSICAL_CHANNEL = alice_bases

In [13]:
def bob_compare_bases(alices_bases, bobs_bases):
    indices = []
    
    for i in range(len(alices_bases)):
        if alices_bases[i] == bobs_bases[i]:
            indices.append(i)
    return indices

In [14]:
agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, bob_bases)

In [15]:
CLASSICAL_CHANNEL = agreeing_bases

## 2.7 Step 7: Find Symmetric Key

Alice and Bob can now discard the bits in their key that used a different encoding and decoding basis.

In [16]:
def key_from_indices(bitstring, indices):
    key = ''
    for idx in indices:
        # For the indices where bases match, the bitstring bit is added to the key
        key = key + bitstring[idx] 
    return key

In [17]:
alice_key = key_from_indices(alice_bitstring, CLASSICAL_CHANNEL)
bob_key = key_from_indices(bob_bitstring, agreeing_bases)

print("alice_key: ", alice_key[:20])
print("bob_key: ", bob_key[:20])
print("Alice's key is equal to Bob's key: ", alice_key == bob_key)

alice_key:  10111100001001111110
bob_key:  10111100001001111110
Alice's key is equal to Bob's key:  True


Finally after Alice and Bob discard every bit that was encoded using a basis that they didn't agree on, they will have a shared key at the end.

In [18]:
BB84_key = alice_key
BB84_key

'1011110000100111111011101101000101010101010000000000010100010111100100101101011111001000110111001010111110010010011001111011110000111100000010001101011010000000100110001110000101000110011100000011010100111001010001111110111100000101000'

# Using the shared key to communicate

In [19]:
import binascii

def encrypt_message(unencrypted_string, key):
    # Convert ascii string to binary string
    bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
    # created the encrypted string using the key
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str( (int(bitstring[i])^ int(key[i])) )
    return encrypted_string
    
def decrypt_message(encrypted_bits, key):
    # created the unencrypted string using the key
    unencrypted_bits = ""
    for i in range(len(encrypted_bits)):
        unencrypted_bits += str( (int(encrypted_bits[i])^ int(key[i])) )
    # Convert bitstring into
    i = int(unencrypted_bits, 2)
    hex_string = '%x' % i
    n = len(hex_string)
    bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
    unencrypted_string = bits.decode('utf-8', 'surrogatepass')
    return unencrypted_string

In [20]:
message = "Quantum Steganography!"
print("Original Messge:", message)
encrypted_message = encrypt_message(message, alice_key)
print("Encrypted message:", encrypted_message)
decrypted_message = decrypt_message(encrypted_message, bob_key)
print("Decrypted message:", decrypted_message)

Original Messge: Quantum Steganography!
Encrypted message: 11101101010100101000111110111111001000010011010101101000001101111100000110100011101011011011101111001110111111000000100011011011010011100110100110100110111010001110000111000000
Decrypted message: Quantum Steganography!


# Embedding the secret message in cover message

In [21]:
def encrypt_cover_msg(encrypted_message, carrier_msg):
    cm = ''
    for i in range(len(encrypted_message)):
        if encrypted_message[i] == '1':
            cm += carrier_msg[i].upper()
        else:
            cm += carrier_msg[i]
    return cm
def decrypt_cover_msg(carrier_msg):
    letters = '' 
    for i in range(len(carrier_msg)):
        if carrier_msg[i].isupper():
            letters+='1'
        else:
            letters+='0'
    return letters

In [22]:
message = "Quantum Steganography!"
print("Original Messge:", message,'\n')
encrypted_message = encrypt_message(message, alice_key)
carrier_msg=''.join([choice("abcdefghijklmnopqrstuvwxyz") for _ in range(len(encrypted_message))])
carrier_msg = encrypt_cover_msg(encrypted_message, carrier_msg)
print("Cover message:", carrier_msg,'\n')
decrypt_msg = decrypt_cover_msg(carrier_msg)
decrypted_message = decrypt_message(decrypt_msg, bob_key)
print("Decrypted message:", decrypted_message)

Original Messge: Quantum Steganography! 

Cover message: NWYrNDhAfFgLdsJiMrbeXTAMSlJRLTMQbbSoolhNizWLjYbFvCIuFzolieCIhVSZOMcpzywGDcJsjeLFJvYoDIdEZiDIRtRPWVkfVUHyMNLTVZqpyqklZtidHFbIQkRNxFkaCJLziMJsLrxPFaZmcMUfWWZvGhncWNJrbwmRFMaqunwo 

Decrypted message: Quantum Steganography!


# Generating new key to communicate

In [29]:
# Step 1
alice_bitstring, alice_bases = select_encoding(key_length)
# Step 2
bob_bases = select_measurement(key_length)
# Step 3
encoded_qubits = encode(alice_bitstring, alice_bases)
# Step 4
QUANTUM_CHANNEL = encoded_qubits
# Step 5
bob_bitstring = measure(bob_bases, QUANTUM_CHANNEL, Aer.get_backend('qasm_simulator'))
# Step 6
CLASSICAL_CHANNEL = alice_bases
agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, bob_bases)
# Step 7
CLASSICAL_CHANNEL = agreeing_bases
alice_key = key_from_indices(alice_bitstring, agreeing_bases)
bob_key = key_from_indices(bob_bitstring, agreeing_bases)

print("alice_key: ", alice_key[:20])
print("bob_key: ", bob_key[:20])
print("Alice's key is equal to Bob's key: ", alice_key == bob_key)

alice_key:  11001010010101000111
bob_key:  11001010010101000111
Alice's key is equal to Bob's key:  True


In [30]:
message = "Quantum Steganography!"
print("Original Messge:", message,'\n')
encrypted_message = encrypt_message(message, alice_key)
carrier_msg=''.join([choice("abcdefghijklmnopqrstuvwxyz") for _ in range(len(encrypted_message))])
carrier_msg = encrypt_cover_msg(encrypted_message, carrier_msg)
print("Cover message:", carrier_msg,'\n')
decrypt_msg = decrypt_cover_msg(carrier_msg)
decrypted_message = decrypt_message(decrypt_msg, bob_key)
print("Decrypted message:", decrypted_message)

Original Messge: Quantum Steganography! 

Cover message: RyxHXrSMqxOewesMskoECnnVgxftFBJjVmMcedKvyvgBojKXwzMFHhEnFzqCiNWjDXIclSnXAvRikhTPOzSTnGNwehMPYuEvJnMbnLCSkwONVmrtvvAydvcHftZWoqASfTdkZdefUnNaKWOpzdZxUftEdPNIHSFZXjdmSMVTQkdlwryb 

Decrypted message: Quantum Steganography!


In [31]:
message = "Hi! I am Mitesh."
print("Original Messge:", message,'\n')
encrypted_message = encrypt_message(message, alice_key)
carrier_msg=''.join([choice("abcdefghijklmnopqrstuvwxyz") for _ in range(len(encrypted_message))])
carrier_msg = encrypt_cover_msg(encrypted_message, carrier_msg)
print("Cover message:", carrier_msg,'\n')
decrypt_msg = decrypt_cover_msg(carrier_msg)
decrypted_message = decrypt_message(decrypt_msg, bob_key)
print("Decrypted message:", decrypted_message)

Original Messge: Hi! I am Mitesh. 

Cover message: EudunzFyjoLQQKrLdTzERnsGqCpemcviKmfGNKGRuRnejSDjwbWKeKBwCFmNOeYJSdoNrXBsOnfONdHrHnUROaTtthNzNeyXSbNcxhMCzjDhsWnFcjNziUWrqWWNDsBk 

Decrypted message: Hi! I am Mitesh.
